In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from random import choices, sample
from gensim.models import Word2Vec
import gc
pd.set_option('display.float_format',lambda x : '%.2f' % x)
from math import log10

In [2]:
with open('./NN_result/lt_df_NN_rank_122days.pkl', 'rb') as file:
    lt_df_NN_rank = pickle.load(file)

In [3]:
full_table = pd.concat(lt_df_NN_rank[15:15+54], ignore_index=True)

In [4]:
full_table['dist'] = full_table.apply(lambda df: df['rank'] - df['NN_rank'], axis=1)

In [5]:
full_table

,target,next,rank,NN_rank,dist
0,102ca9ccb86811e9b0f5acde48001122,e360cdcab86611e987efacde48001122,10.00,7,3.00
1,e360cdcab86611e987efacde48001122,06fb62c2b86711e9867dacde48001122,4.00,4,0.00
2,baaadc26b86711e9b9cbacde48001122,e353a26cb86611e992e5acde48001122,60.00,48,12.00
3,e353a26cb86611e992e5acde48001122,e35fafb4b86611e99d1dacde48001122,59.00,20,39.00
4,e336d7e2b86611e9a7f7acde48001122,e3875b18b86611e9bab5acde48001122,40.00,28,12.00
...,...,...,...,...,...
2176234,17a1fbf4b86711e98eb1acde48001122,1794c89eb86711e99a2eacde48001122,3.00,2,1.00
2176235,1794c89eb86711e99a2eacde48001122,5ba15890b86711e9ab99acde48001122,1.00,1,0.00
2176236,e33344e8b86611e98696acde48001122,e36092b0b86611e98917acde48001122,6.00,1,5.00
2176237,e34457bab86611e9a371acde48001122,17aee940b86711e9a801acde48001122,7.00,20,-13.00


In [6]:
full_table.describe()

,rank,NN_rank,dist
count,2176239.00,2176239.00,2176239.00
mean,20.54,20.74,-0.21
std,24.51,22.74,24.87
min,1.00,1.00,-98.00
25%,3.00,4.00,-10.00
50%,9.00,11.00,0.00
75%,30.00,31.00,8.00
max,99.00,99.00,98.00


In [7]:
table_grby = full_table.groupby(['target'])
pd.DataFrame(table_grby.size())

,0
target,
008f3dccb86811e9b6c1acde48001122,5
008f3e76b86811e98317acde48001122,11
008f42e8b86811e9bd2aacde48001122,3
008f4862b86811e9a907acde48001122,2
008f49cab86811e98f31acde48001122,9
...,...
f94557e6b86b11e98019acde48001122,1
f9456918b86b11e9b213acde48001122,1
f945b88ab86b11e99cacacde48001122,3


In [8]:
df_table = pd.merge(full_table, pd.DataFrame(table_grby.size()), on=['target'], how='left')

In [9]:
df_table.rename(columns = {0:'occur'}, inplace=True)

In [10]:
# df_table.drop_duplicates(inplace=True, ignore_index=True)

In [11]:
df_table

,target,next,rank,NN_rank,dist,occur
0,102ca9ccb86811e9b0f5acde48001122,e360cdcab86611e987efacde48001122,10.00,7,3.00,45
1,e360cdcab86611e987efacde48001122,06fb62c2b86711e9867dacde48001122,4.00,4,0.00,134
2,baaadc26b86711e9b9cbacde48001122,e353a26cb86611e992e5acde48001122,60.00,48,12.00,48
3,e353a26cb86611e992e5acde48001122,e35fafb4b86611e99d1dacde48001122,59.00,20,39.00,38
4,e336d7e2b86611e9a7f7acde48001122,e3875b18b86611e9bab5acde48001122,40.00,28,12.00,9
...,...,...,...,...,...,...
2176234,17a1fbf4b86711e98eb1acde48001122,1794c89eb86711e99a2eacde48001122,3.00,2,1.00,7
2176235,1794c89eb86711e99a2eacde48001122,5ba15890b86711e9ab99acde48001122,1.00,1,0.00,10
2176236,e33344e8b86611e98696acde48001122,e36092b0b86611e98917acde48001122,6.00,1,5.00,442
2176237,e34457bab86611e9a371acde48001122,17aee940b86711e9a801acde48001122,7.00,20,-13.00,20


In [12]:
df_table.describe()

,rank,NN_rank,dist,occur
count,2176239.00,2176239.00,2176239.00,2176239.00
mean,20.54,20.74,-0.21,112.07
std,24.51,22.74,24.87,371.26
min,1.00,1.00,-98.00,1.00
25%,3.00,4.00,-10.00,8.00
50%,9.00,11.00,0.00,22.00
75%,30.00,31.00,8.00,71.00
max,99.00,99.00,98.00,4931.00


In [13]:
df_table.groupby('occur').mean().describe()

,rank,NN_rank,dist
count,496.00,496.00,496.00
mean,18.71,16.30,2.42
std,6.17,5.25,5.77
min,1.00,2.80,-46.00
25%,14.97,12.94,0.28
50%,19.21,16.48,2.11
75%,21.96,19.70,4.05
max,47.39,47.00,39.70


---

In [25]:
target_grby = table_grby # df_table.groupby(['target'])

In [26]:
target_array = df_table['target'].unique()
training_table = pd.DataFrame({'target': target_array})
training_table = pd.merge(training_table, pd.DataFrame(table_grby.size()), on=['target'], how='left')
training_table.rename(columns = {0:'occur'}, inplace=True)

In [27]:
training_table['rank'] = training_table.apply(lambda df: target_grby.get_group(df['target'])['rank'].mean(), axis=1)
training_table['NN_rank'] = training_table.apply(lambda df: target_grby.get_group(df['target'])['NN_rank'].mean(), axis=1)
training_table['label'] = training_table.apply(lambda df: df['rank'] > df['NN_rank'], axis=1)
training_table['dist'] = training_table.apply(lambda df: target_grby.get_group(df['target'])['dist'].mean(), axis=1)

In [28]:
training_table

,target,occur,rank,NN_rank,label,dist
0,102ca9ccb86811e9b0f5acde48001122,45,14.91,10.18,True,4.73
1,e360cdcab86611e987efacde48001122,134,9.13,6.78,True,2.34
2,baaadc26b86711e9b9cbacde48001122,48,16.96,33.44,False,-16.48
3,e353a26cb86611e992e5acde48001122,38,24.16,28.97,False,-4.82
4,e336d7e2b86611e9a7f7acde48001122,9,28.33,20.67,True,7.67
...,...,...,...,...,...,...
252458,09bc7246b86a11e99a9dacde48001122,1,1.00,1.00,False,0.00
252459,f8297b4ab86a11e9a645acde48001122,1,1.00,1.00,False,0.00
252460,6c589d58b86711e9a4f2acde48001122,1,6.00,4.00,True,2.00
252461,6c60ab54b86711e987faacde48001122,1,1.00,1.00,False,0.00


In [25]:
training_table['label'].value_counts()

False    149030
True     103433
Name: label, dtype: int64

In [22]:
def trans(df):
    if df['label'] and df['occur']==1:
        if df['dist'] < 25:
            return False
    return df['label']

In [23]:
training_table['label_x'] = training_table.apply(trans , axis=1)

In [24]:
training_table

,target,occur,rank,NN_rank,label,dist,label_x
0,102ca9ccb86811e9b0f5acde48001122,45,14.91,10.18,True,4.73,True
1,e360cdcab86611e987efacde48001122,134,9.13,6.78,True,2.34,True
2,baaadc26b86711e9b9cbacde48001122,48,16.96,33.44,False,-16.48,False
3,e353a26cb86611e992e5acde48001122,38,24.16,28.97,False,-4.82,False
4,e336d7e2b86611e9a7f7acde48001122,9,28.33,20.67,True,7.67,True
...,...,...,...,...,...,...,...
252458,09bc7246b86a11e99a9dacde48001122,1,1.00,1.00,False,0.00,False
252459,f8297b4ab86a11e9a645acde48001122,1,1.00,1.00,False,0.00,False
252460,6c589d58b86711e9a4f2acde48001122,1,6.00,4.00,True,2.00,False
252461,6c60ab54b86711e987faacde48001122,1,1.00,1.00,False,0.00,False


In [26]:
training_table['label_x'].value_counts()

False    165252
True      87211
Name: label_x, dtype: int64

---

In [31]:
df_table_testday = pd.concat(lt_df_NN_rank[15+54:], ignore_index=True)

In [32]:
df_table_testday.drop_duplicates(inplace=True, ignore_index=True)

In [33]:
df_table_testday.describe()

,rank,NN_rank
count,810982.00,810982.00
mean,25.83,26.63
std,26.32,24.95
min,1.00,1.00
25%,5.00,6.00
50%,15.00,18.00
75%,40.00,41.00
max,99.00,99.00


In [34]:
switch_table = training_table.drop(columns=['rank', 'NN_rank', 'dist'])

In [35]:
df_table_testday_switch = pd.merge(df_table_testday, switch_table, on=['target'], how='left')

In [36]:
from multiprocessing import  Pool
def parallelize_dataframe(df, func, n_cores=8):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [37]:
def add_features(df_table_testday_switch):
    df_table_testday_switch['Ground_Truth'] = df_table_testday_switch.apply(lambda df: df['rank'] > df['NN_rank'], axis=1)
    df_table_testday_switch['switch'] = df_table_testday_switch.apply(lambda df: df['NN_rank'] if df['label'] else df['rank'], axis=1)
    df_table_testday_switch['switch_fillna_NN'] = df_table_testday_switch.apply(lambda df: df['NN_rank'] if np.isnan(df['label']) else df['switch'], axis=1)
    df_table_testday_switch['switch_fillna_knn'] = df_table_testday_switch.apply(lambda df: df['rank'] if np.isnan(df['label']) else df['switch'], axis=1)
#     df_table_testday_switch['switch_x'] = df_table_testday_switch.apply(lambda df: df['NN_rank'] if df['label_x'] else df['rank'], axis=1)
    return df_table_testday_switch

In [38]:
df_table_testday_switch = parallelize_dataframe(df_table_testday_switch, add_features)

In [39]:
df_table_testday_switch

,target,next,rank,NN_rank,occur,label,Ground_Truth,switch,switch_fillna_NN,switch_fillna_knn
0,e3390966b86611e9bb8facde48001122,8bf8b80cb86811e9972dacde48001122,16.00,35,175.00,False,False,16.00,16.00,16.00
1,8bf8b80cb86811e9972dacde48001122,f5adf298b86611e9930dacde48001122,11.00,13,106.00,True,False,13.00,13.00,13.00
2,f5adf298b86611e9930dacde48001122,f5accde4b86611e9acc7acde48001122,80.00,68,114.00,True,True,68.00,68.00,68.00
3,f5accde4b86611e9acc7acde48001122,571cafe2b86c11e99481acde48001122,1.00,40,9.00,False,False,1.00,1.00,1.00
4,571cafe2b86c11e99481acde48001122,8bf8b80cb86811e9972dacde48001122,36.00,58,1.00,False,False,36.00,36.00,36.00
...,...,...,...,...,...,...,...,...,...,...
810977,56dd0c00b86c11e993e3acde48001122,c2d5056eb86911e9b888acde48001122,65.00,12,nan,NaN,True,12.00,12.00,65.00
810978,470a8500b86711e9a854acde48001122,470a8e88b86711e9853aacde48001122,25.00,9,27.00,True,True,9.00,9.00,9.00
810979,06f92b38b86711e9a019acde48001122,070838e4b86711e98c7bacde48001122,8.00,13,15.00,True,False,13.00,13.00,13.00
810980,e376ed6cb86611e982b2acde48001122,0714dd42b86711e988c6acde48001122,16.00,37,37.00,True,False,37.00,37.00,37.00


In [40]:
df_table_testday_switch[df_table_testday_switch['label'].isnull()]

,target,next,rank,NN_rank,occur,label,Ground_Truth,switch,switch_fillna_NN,switch_fillna_knn
32,f592d6b4b86611e995dcacde48001122,2f5e4dccb86711e98f55acde48001122,20.00,20,nan,NaN,False,20.00,20.00,20.00
49,e36c3c9eb86611e9bf1facde48001122,1791656eb86711e9b707acde48001122,14.00,42,nan,NaN,False,42.00,42.00,14.00
86,073b09a4b86711e9a445acde48001122,5c052686b86711e9851dacde48001122,42.00,60,nan,NaN,False,60.00,60.00,42.00
240,07159fc0b86711e999e6acde48001122,07159f3eb86711e99bd3acde48001122,1.00,13,nan,NaN,False,13.00,13.00,1.00
250,5c04e6dab86711e995e5acde48001122,2f5c47c2b86711e99d2cacde48001122,9.00,10,nan,NaN,False,10.00,10.00,9.00
...,...,...,...,...,...,...,...,...,...,...
810931,e37781fab86611e9bcc4acde48001122,472b9d6cb86711e98a65acde48001122,21.00,94,nan,NaN,False,94.00,94.00,21.00
810932,472b9d6cb86711e98a65acde48001122,072c9450b86711e9a8adacde48001122,45.00,57,nan,NaN,False,57.00,57.00,45.00
810948,16ba9ae2b86d11e985c0acde48001122,e33b6012b86611e99398acde48001122,33.00,8,nan,NaN,True,8.00,8.00,33.00
810977,56dd0c00b86c11e993e3acde48001122,c2d5056eb86911e9b888acde48001122,65.00,12,nan,NaN,True,12.00,12.00,65.00


In [41]:
df_table_testday_switch.describe()

,rank,NN_rank,occur,switch,switch_fillna_NN,switch_fillna_knn
count,810982.00,810982.00,718160.00,810982.00,810982.00,810982.00
mean,25.83,26.63,31.38,24.89,24.89,24.47
std,26.32,24.95,83.93,24.93,24.93,24.86
min,1.00,1.00,1.00,1.00,1.00,1.00
25%,5.00,6.00,4.00,5.00,5.00,5.00
50%,15.00,18.00,11.00,15.00,15.00,15.00
75%,40.00,41.00,28.00,38.00,38.00,37.00
max,99.00,99.00,4931.00,99.00,99.00,99.00


In [42]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(df_table_testday_switch['Ground_Truth'].tolist(),df_table_testday_switch['label'].fillna(True).tolist()))

              precision    recall  f1-score   support

       False       0.61      0.49      0.54    452057
        True       0.48      0.61      0.54    358925

    accuracy                           0.54    810982
   macro avg       0.55      0.55      0.54    810982
weighted avg       0.55      0.54      0.54    810982

